In [ ]:
# Import packages
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import os

os.system("mkdir unprocessed")

In [ ]:
# Load the list of all tickers to query
all_tickers_list = []
for line in open("ticker_labels.txt"):
  all_tickers_list.append(line.replace("\n", ""))
all_tickers = list(np.unique(all_tickers_list))
all_tickers.sort()

In [ ]:
# Download data for all tickers

#tickers = ["A", "AA", "AADI", "AAGR", "AAIC", "AAL", "AAM", "AAMC", "AAME"]
tickers = all_tickers

tickers_have = []
tickers_no_data = []

all_data = None
for t in tickers:
  data = yf.download(t, start="2020-01-01")
  data.columns = data.columns.droplevel(1)
  processed_data = data[["Close", "Volume"]]
  if processed_data["Close"].shape == (0,):
    usable = False
  else:
    usable = True
  new_names = {"Close": t+"_close", "Volume": t+"_volume"}
  processed_data = processed_data.rename(new_names, axis=1)
  if type(all_data) == type(None) and usable:
    all_data = processed_data
    tickers_have.append(t)
  else:
    if usable:
      all_data = all_data.join(processed_data)
      tickers_have.append(t)
    else:
      tickers_no_data.append(t)

In [ ]:
# Save the data
all_data.to_csv("unprocessed/stock_price_data.csv")